### 一. Basic environment initialization

#### 1. Initialize the basic environment

In [ ]:
%%bash
sudo chmod +x init-env.sh
./init-env.sh

### 二. Image and Snapshot preparation

#### 1. Mirroring(~15min)

In [ ]:
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
os.chdir('/home/ec2-user/SageMaker/stable-diffusion-on-eks/deploy/tools')
cmd = "/bin/sh ./build_and_push.sh %s" %(region_name)
os.system(cmd)

After the above command is executed, please check whether there is a corresponding repository and image in ECR.

#### 2. Snapshot production (this step is optional, it is recommended to do it after the business code development and debugging is completed) (~20min)

In [ ]:
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
Infer_API_IMG= "%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/inference-api:latest" %(account_id,region_name)
Queue_Agent_IMG= "%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/queue-agent:latest" %(account_id,region_name)
OCI_URL = "oci://%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/charts/" %(account_id,region_name)
print ("OCI_URL is :",OCI_URL)
os.chdir('/home/ec2-user/SageMaker/stable-diffusion-on-eks/utils/bottlerocket-images-cache')
cmd = "/bin/sh ./snapshot.sh %s %s" %(Infer_API_IMG,Queue_Agent_IMG)
os.system(cmd)

In [ ]:
#Record the snapshot id output above

### 三. CDK deployment

#### Modify the config.yaml file, refer to editing the config.yaml file in the Sample directory, pay attention to punctuation and format

Modify according to the actual situation, model S3 bucket arn, runtime model, self-built ECR, Snapshot ID (if enabled), etc. Save after editing

In [ ]:
#Already echo "export s3uri=${S3Bucket}" in NotebookInstanceLifecycleConfig >> /etc/profile.d/jupyter-env.sh
! echo "arn:aws:s3:::$s3uri"

#### Perform CDK deployment (~20min)

In [ ]:
import os
os.chdir('/home/ec2-user/SageMaker/stable-diffusion-on-eks/deploy/')
cmd = "/bin/sh ./cdk_deploy.sh"
os.system(cmd)

In [10]:
#Wait for cloudformation execution to complete
!aws cloudformation describe-stacks --stack-name SdOnEksStack --output json --query 'Stacks[0].StackStatus'

"CREATE_COMPLETE"


After the execution is completed, the APIGateway address is output from cloudformation.

In [13]:
APIGW_URL="$(aws cloudformation describe-stacks --stack-name SdOnEksStack --output text --query 'Stacks[0].Outputs[?OutputKey==`FrontApiEndpoint`].OutputValue')"

In [ ]:
!echo "APIGW URL:" $APIGW_URL

After execution, if you need to manage EKS, execute the following command, output the record eks config update command, and copy it to bash for execution.

In [ ]:
!aws cloudformation describe-stacks --stack-name SdOnEksStack --output text --query 'Stacks[0].Outputs[?OutputKey==`ConfigCommand`].OutputValue'

### 4. Model download synchronization

#### 1. Model download (time required varies depending on model size)

Find the S3uploader folder in the tools directory and modify the down.csv in it.
Modify the model download url and target key column (strictly follow the prefix/key structure) and save it.

Among them, prefix allowed values: "stable-diffusion", "lora", "vae", "embeddings"

In [ ]:
%%bash
cd /home/ec2-user/SageMaker/stable-diffusion-on-eks/deploy/tools/S3uploader
sed -i "s/\,\,/\,$s3uri,/g" down.csv
python3 s3uploader.py down.csv

#### 2. Model synchronization

Enter the datasync service access console to find out whether there are execution records in the history.

#### 5. Request a test

In [ ]:
## Get API URL

In [28]:
!echo "API URL:" $APIGW_URL

API URL: https://fy0rwyomqi.execute-api.us-west-2.amazonaws.com/prod/


In [ ]:
## Get API key

In [31]:
getkey="$(aws cloudformation describe-stacks --stack-name SdOnEksStack --output text --query 'Stacks[0].Outputs[?OutputKey==`GetAPIKeyCommand`].OutputValue')"

In [44]:
!$getkey

None
